## Tasks to be completed here for airlines assistant
1. availability tool & booking tool
2. add an agent to translate all the responses in preferred language - use claude here
3. add another agent that listens to audio and converts into text
4. use another agent that generates image - use GPT here.

In [1]:
## imports
import os
from dotenv import load_dotenv
import openai
import anthropic
import gradio as gr
import json
import pandas as pd
import random, string

import base64          # base64 endoding/decoding
from io import BytesIO # In-Memory binary file - writes in RAM
from PIL import Image  # Image utility tool

from pydub import AudioSegment
from pydub.playback import play

from dateutil import parser as dateparser

In [2]:
load_dotenv()

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY=os.getenv('ANTHROPIC_API_KEY')


MODEL = 'gpt-4o-mini'
openai_client = openai.OpenAI()
claude_client = anthropic.Anthropic()

In [3]:
## configs
OPENAI_CHAT_MODEL      = "gpt-4o-mini"
OPENAI_TTS_MODEL       = "tts-1"
OPENAI_TTS_VOICE       = "alloy"        # onyx/alloy/verse 
OPENAI_ASR_MODEL       = "whisper-1"    
OPENAI_IMAGE_MODEL     = "dall-e-3" 
PREFERRED_LANGUAGE     = "Bengali"      # change anytime
CLAUDE_TRANSLATION_MODEL = "claude-3-7-sonnet-latest"

BOOKINGS_DB = {}

DATA_PATH = "flights.csv"

SYSTEM_PROMPT = f"""
You are a very helpful assistant for an airline called SydAirAI.
Always answer in 1 sentence. If unsure, say so plainly.

Before calling any tool, NORMALIZE user inputs:
- Map common city names to IATA codes (Sydney→SYD, Melbourne→MEL, Brisbane→BNE, Perth→PER, Adelaide→ADL, Canberra→CBR).
- Uppercase IATA codes.
- Convert any date to YYYY-MM-DD.
- If the user says 'any' cabin or doesn't specify cabin, use 'any' for cabin.
- If seats not specified, assume 1.

If a city is ambiguous (e.g., “Newcastle”), ask a clarifying question.
"""

In [4]:
CITY_TO_IATA = {
    "sydney": "SYD", "syd":"SYD",
    "melbourne": "MEL", "mel": "MEL", "MELB": "MEL",
    "brisbane":"BNE", "bne":"BNE",
}

def to_iata(city):
    if not city:
        return city
    key = city.strip().lower()
    return CITY_TO_IATA.get(key, city.strip().lower())

def to_iso_date(date):
    return dateparser.parse(date, fuzzy=True).strftime("%Y-%m-%d")

## load the csv data
data = pd.read_csv(DATA_PATH)

## generic funtions to call the tools

## tool 1: availability of the files
def check_availability(
    origin: str,
    destination: str,
    date: str,
    cabin: str,
    required_seats: int = 1,
    max_results: int=3
):
    try:
        ## normalize the input values
        origin = to_iata(origin)
        destination = to_iata(destination)
        date = to_iso_date(date)
        cabin = (cabin or "").strip().lower()
        required_seats = int(required_seats)

        ## query the data
        df = data.query("origin == @origin and destination == @destination and date == @date and seats_left >= @required_seats")

        ## if cabin is provided
        if cabin and cabin !="any":
            df = df[df["cabin"].str.lower() == cabin]

        ## sort by the cheapest and earliest departure
        df = df.sort_values(["price", "depart"]).head(max_results)
    
        ## if search result is empty
        if df.empty:
            return {
                "ok": False,
                "message": "No flight found at this moment"
            }
        return {
            "ok": True,
            "flights": df.to_dict(orient="records")
        }
    except Exception as e:
        return{
            "ok":False,
            "error": str(e)
        }



## tool 2: Book a flight

## in-memory storage for booking
bookings = {}

def book_flight(
    flight_no: str, 
    name: str, 
    email: str, 
    date: str,
    seats_asked: int,
    cabin: str
):
    try:
        flight_no = flight_no.strip().upper()
        name = name.strip()
        email = email.strip()
        date = to_iso_date(date)
        seats_asked = int(seats_asked)
        cabin = (cabin or "").strip().lower()

        ## guardrail if asked_seat is minus or zero
        if seats_asked <= 0:
            return{
                "ok":False,
                "message": "seats should be for at least 1 person."
            }
            
        ## find the flight
        df = data.query("flight_no == @flight_no and date == @date")
        if df.empty:
            return {
                "ok": False,
                "message": "No flights found with the search criteria"
            }

        row = df.iloc[0]
            
        ## availability check
        if cabin and row["cabin"].strip().lower() != cabin:
            return{
                "ok":False,
                "message": f"Cabin mismatch. Flight cabin is '{row['cabin']}'."
            }
        
        seats_left = int(row["seats_left"])
        if seats_left<seats_asked:
            return {
                "ok":False,
                "message": "Not enough seats available"
            }
            
        ## reduce the seats_left once booked - main data
        idx = data.index[(data.flight_no == flight_no) & (data.date==date)][0]
        data.at[idx, "seats_left"] = int(data.at[idx, "seats_left"]) - int(seats_asked)

        ## generate a sample PNR
        pnr = "".join(random.choices(string.ascii_uppercase +  string.digits, k=6))

        ## save the booking
        booking = {
            "pnr": pnr,
            "flight_no":flight_no,
            "date": date,
            "origin": row.origin,
            "destination": row.destination,
            "depart": row.depart,
            "arrive": row.arrive,
            "cabin": row.cabin,
            "name": name,
            "email": email,
            "seats": seats_asked,
            "price_total": float(row.price) * seats_asked
        }
        bookings[pnr] = booking

        return {
            "ok":True,
            "booking": booking
        }
        
    except Exception as e:
        return {
            "ok":False,
            "error": str(e)
        }

In [5]:
## schemas

## following the particular dictionary structure to describe the functions as tools

## tool registration - 1: Availability tool
availability_tool = {
    "name": "check_availability",
    "description": """Check flight availability. Always convert city names to IATA codes
                      (e.g., Sydney→SYD, Melbourne→MEL) and pass those to the tool.
                      Convert dates to YYYY-MM-DD. If the customer says 'any' cabin or
                      doesn't specify, pass cabin='any'. Assume 1 seat if not given.
                      The customer might ask 'I want to travel from sydney to melbourne on 22 of july in economy call. I am just travelling by myself'""",
    "parameters":{
        "type": "object",
        "properties":{
            "origin":{
                "type": "string",
                "description": "The city the customer is travelling from"
            },
            "destination":{
                "type": "string",
                "description": "The city the customer wants to go to"
            },
            "date":{
                "type":"string",
                "description": "The date the customer is willing to travel from the origin city"
            },
            "cabin":{
                "type": "string",
                "description": "The type of cabin the customer is willing to travel in. for example: business and economy"
            },
            "required_seats":{
                "type": "integer",
                "description": "The number of seats the customer wants to book"
            },
            "max_results":{
                "type":"integer",
                "description": "the maximum number of response that can be obtained from a search with the check_availability function"
            },
        },
        "required":["origin", "destination", "date", "cabin", "required_seats"],
        "additionalProperties": False
    },
}

## tool registration - 2: Booking tool
booking_tool = {
    "name": "book_flight",
    "description": """ Book the flight of the customer when they choose a specific flight.
                       call this function when the customer confirms that they want to make a reservation.
                       this requires a flight number, travel date, passenger name and email.
                       reserve the seats with cabin type.
                       Example: I want to book 2 economy tickets on AB123 for july 22 under Mostafa Protik""",
    "parameters":{
        "type": "object",
        "properties":{
            "flight_no":{
                "type": "string",
                "description": "the selected flight for the customer to travel"
            },
            "name":{
                "type": "string",
                "description": "The name of the customer under that he/she will be travelling"
            },
            "email":{
                "type":"string",
                "description": "The email address of the customer that is used in booking"
            },
            "date":{
                "type": "string",
                "description": "The date of booking when the customer wants to travel"
            },
            "seats_asked":{
                "type": "integer",
                "description": "The total number of seats the customer wants to reserve"
            },
            "cabin":{
                "type":"string",
                "description": "the type of cabin the customer wishes to reserve"
            },
        },
        "required":["flight_no", "name", "email", "date", "seats_asked", "cabin"],
        "additionalProperties": False
    },
}


## tools registration
tools = [
    {"type": "function", "function": availability_tool},
    {"type": "function", "function": booking_tool}
]

In [6]:
def artist(city):
    image_response = openai_client.images.generate(
        model = "dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about the {city}, in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [7]:
def talker(message):
    response = openai_client.audio.speech.create(
        model = "tts-1",
        voice = "onyx",
        input = message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [8]:
def _run_tool(name, args):
    if name=="check_availability":
        return check_availability(**args)
    if name=="book_flight":
        return book_flight(**args)
    return {
        "ok": False,
        "message": f"unknown tool: {name}"
    }

In [9]:
## tool layer
def handle_tool_call(message):
    tool_messages = []
    booking_city = None
    ## looping over all the requested tools
    for tool_call in message.tool_calls:
        name = tool_call.function.name # get the name of the tool
        args = json.loads(tool_call.function.arguments) # get the args to run the function
        result = _run_tool(name, args) # run the function in python
        tool_messages.append({
            "role":"tool",
            "tool_call_id": tool_call.id,
            "name": name,
            "content": json.dumps(result)
        })
        print(f"TOOL ARGS: {name}\n\n{args}")
        if name=="book_flight" and result.get("ok") and "booking" in result:
            booking_city = result["booking"].get("destination")
    return tool_messages, booking_city

In [10]:
def chat(message, history):
    messages = [
        {"role":"system", "content": SYSTEM_PROMPT}
    ]

    for human, assistant in history:
        messages.append({"role":"user", "content": human})
        messages.append({"role":"assistant", "content": assistant})
    messages.append({"role": "user", "content": message})

    response = openai_client.chat.completions.create(
        model = MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    image = None
    
    ## if tool is required - not the final answer, rather an instruction to call the tools
    if response.choices[0].finish_reason=="tool_calls":
        ## capture assistants request
        assistant_message = response.choices[0].message
        # messages.append({"role":"assistant", "tool_calls": assistant_message.tool_calls})
        messages.append(assistant_message)
        ## run tool manually and append outputs
        tools_msgs, booking_city = handle_tool_call(assistant_message)
        messages.extend(tools_msgs)

        ## finalize
        response = openai_client.chat.completions.create(
            model = MODEL,
            messages = messages
        )
        reply = response.choices[0].message.content
        try: 
            talker(reply)
        except Exception as e: 
            print(f"[TTS suppressed] {e}")

        if booking_city:
            image = artist(booking_city)   
        return reply, image

    # no tool call path
    reply = response.choices[0].message.content
    try:
        talker(reply)
    except Exception as e:
        print(f"[TTS error]: {e}")
    return reply

In [11]:
# more involved gradio code as we are not using off-the-shelf class

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500, type="pil")

    with gr.Row():
        msg = gr.Textbox(label = "Chat with our AI assistant: ")

    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        result = chat(user_message, history[:-1])  # unpacking can be just text or text and image

        ## handling the cases
        if isinstance(result, tuple):
            reply, img = result[0], result[1]
        else:
            reply, img = result, None
        history[-1][1] = reply
        return history, img

    msg.submit(
        user,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot,
        chatbot,
        [chatbot, image_output]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

ui.launch()

/var/folders/x7/xysp_t5n7pz6r9vs9x6sjsxm0000gn/T/ipykernel_6414/1745871388.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [12]:
def translate_to_preferred_language(
    text: str,
    preferred_language: str = PREFERRED_LANGUAGE
):
    try:
        response = claude_client.messages.create(
            model = CLAUDE_TRANSLATION_MODEL,
            max_tokens = 1000,
            system = f"""You are a very helpful assistant in translating one language to another.
                         Always translate English to {preferred_language}""",
            messages = [
                {"role":"user", "content": text}
            ]
        )
        return response.content[0].text.strip()
    except Exception as e:
        print(f"Tranlator failed: {e}")
        return text

In [13]:
def chat(message, history):
    messages = [
        {"role":"system", "content": SYSTEM_PROMPT}
    ]

    for human, assistant in history:
        messages.append({"role":"user", "content": human})
        messages.append({"role":"assistant", "content": assistant})
    messages.append({"role": "user", "content": message})

    response = openai_client.chat.completions.create(
        model = MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    image = None
    
    ## if tool is required - not the final answer, rather an instruction to call the tools
    if response.choices[0].finish_reason=="tool_calls":
        ## capture assistants request
        assistant_message = response.choices[0].message
        # messages.append({"role":"assistant", "tool_calls": assistant_message.tool_calls})
        messages.append(assistant_message)
        ## run tool manually and append outputs
        tools_msgs, booking_city = handle_tool_call(assistant_message)
        messages.extend(tools_msgs)

        ## finalize
        response = openai_client.chat.completions.create(
            model = MODEL,
            messages = messages
        )
        reply = response.choices[0].message.content
        reply_tr = translate_to_preferred_language(reply)
        try: 
            talker(reply_tr)
        except Exception as e: 
            print(f"[TTS suppressed] {e}")

        if booking_city:
            image = artist(booking_city)   
        return reply, image

    # no tool call path
    reply = response.choices[0].message.content
    reply_tr = translate_to_preferred_language(reply)
    try:
        talker(reply_tr)
    except Exception as e:
        print(f"[TTS error]: {e}")
    return reply, image

In [14]:
# more involved gradio code as we are not using off-the-shelf class

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500, type="pil")

    with gr.Row():
        msg = gr.Textbox(label = "Chat with our AI assistant: ")

    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        result = chat(user_message, history[:-1])  # unpacking can be just text or text and image

        ## handling the cases
        if isinstance(result, tuple):
            reply, img = result[0], result[1]
        else:
            reply, img = result, None
        history[-1][1] = reply
        return history, img

    msg.submit(
        user,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot,
        chatbot,
        [chatbot, image_output]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

ui.launch()

/var/folders/x7/xysp_t5n7pz6r9vs9x6sjsxm0000gn/T/ipykernel_6414/1745871388.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [15]:
# turning audio to text
def transcribe_to_text(audio_path):
    try:
        with open(audio_path, "rb") as f:
            response = openai_client.audio.transcriptions.create(
                model = OPENAI_ASR_MODEL,
                file = f,
                response_format = "text"
            )
        return response.strip()
    except Exception as e:
        print(f"ASR Failed: {e}")
        return ""

In [16]:
## bridging mic to text chat
def voice_user(audio_path, history):
    text = transcribe_to_text(audio_path) or "voice was not audible"
    history.append([text, None]) ## it is a [user, assistant] pair. we are just putting the use text here.
    return None, history

In [17]:
# adding the microphone input in the previous UI

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500, type="pil")

    with gr.Row():
        msg = gr.Textbox(label = "Chat with our AI assistant: ")
        mic = gr.Audio(sources=["microphone"], type="filepath")

    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        result = chat(user_message, history[:-1])  # unpacking can be just text or text and image

        ## handling the cases
        if isinstance(result, tuple):
            reply, img = result[0], result[1]
        else:
            reply, img = result, None
        history[-1][1] = reply
        return history, img

    msg.submit(
        user,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot,
        chatbot,
        [chatbot, image_output]
    )

    ## mic componenet
    
    mic.stop_recording(
        voice_user,
        [mic, chatbot],
        [mic, chatbot],
        queue = False
    ).then(
        bot, 
        chatbot,
        [chatbot, image_output]
    )

    clear.click(
        lambda: ([], None, None, ""), 
        None, 
        [chatbot, image_output, mic, msg], 
        queue=False
    )

ui.launch()

/var/folders/x7/xysp_t5n7pz6r9vs9x6sjsxm0000gn/T/ipykernel_6414/1028851862.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
